# Third session - To start solving the fourth tasks of the proposed project!

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The last task, the one that we will devote the most of our time, is to use the dataset to perform an activtity recognition task. That means that, given a sample of the dataset your code must be able to recognize from which of the 10 types of activity it came from, and the accuracy should be measurable beyond simple showing the confusion matrix.

Following the results from the previous session, we should have a complete segmented dataset. The next step is to group all activities in a single dataset and label them the name of the activities in the activites file. This could be done by sligthly altering the code used to segment the data adding the correponding number of the activity to the last column. The order of the activities here.

As a suggesstion, if you used the code from the previous notebooks as a start to your segmentation, the modification to the code can be as simple as the piece of code below:

In [ ]:
# DataFrame = pd.read_csv('/content/drive/MyDrive/projet_data_science_m2/projet_data/data/mergeData.csv', sep=',')

In [ ]:
def segmentActivity(base, calendar):
    labelled_dataset = pd.DataFrame()
    activity_name = calendar['activity'].drop_duplicates().tolist()
    # Itère sur chaque activité dans le calendrier pour traiter les intervalles de temps.
    for labelNumber in range(len(activity_name)):
        for idact, act in enumerate(calendar['activity']):
            start = calendar['Started'][idact].tz_convert('UTC+01:00')
            end = calendar['Ended'][idact].tz_convert('UTC+01:00')
            if act == activity_name[labelNumber]:
                act1new = base[(base['Time'] >= start) & (base['Time'] <= end)].reset_index(drop=True).sort_values(
                    by='Time').drop(columns='Time').dropna().reset_index(drop=True)
                act1new['label'] = labelNumber * np.ones(len(act1new))  # <<--- Adds the label as the last column
                labelled_dataset = pd.concat([labelled_dataset, act1new], ignore_index=True)

    return labelled_dataset

In [ ]:
mergeData = pd.read_csv('/content/drive/MyDrive/projet_data_science_m2/projet_data/data/mergeData.csv', sep=',')
mergeData['Time'] = pd.to_datetime(mergeData['Time']).dt.tz_convert('UTC+01:00')
print("mergeData ...DONE")

# On charge le fichier activities.xlxs avec les 3 colonnes souhaitées : 'activity', 'Started', 'Ended'
calendar = pd.read_excel('/content/drive/MyDrive/projet_data_science_m2/TP_Data_2024FISA/activites.xlsx', sheet_name='Done so far', usecols=['activity', 'Started', 'Ended'])
calendar['Started'] = pd.to_datetime(calendar['Started']).dt.tz_localize('UTC').dt.tz_convert('UTC+01:00')
calendar['Ended'] = pd.to_datetime(calendar['Ended']).dt.tz_localize('UTC').dt.tz_convert('UTC+01:00')
# On enleve les cellules 'NaN'
calendar = calendar.dropna().reset_index(drop=True)
print("calendar ...DONE")

# Segmentation des activités qui sont concaténées dans un résultat affecté à labelled_dataset
labelled_dataset = segmentActivity(mergeData, calendar)
print("segmentActivity ...DONE")

# Sauvegarde du jeu de données dans un fichier .csv
output = "/content/drive/MyDrive/projet_data_science_m2/projet_data/data/labelled_dataset.csv"
labelled_dataset.to_csv(output, index=False, sep=';')
print("create data file ...DONE")

mergeData ...DONE
calendar ...DONE
segmentActivity ...DONE
create data file ...DONE


Them you concatenate all activities and save it to a .csv file that is going to be loaded into the pytorch environment in which the machine learning is going to take place.

Then, you can the codes provided from the previous TPs to create your machine learning layers.

The suggestions in the presentation were: K-NN, LSTM, and CNNs, but you can use whatever neural network architecture (even a simple feedforward neural net) you want, as long as the classification result is measurable and the process well explained.

In [ ]:
class CustomDataset(Dataset):
    # X représente les données du jeu de données (hors labels)
    # y représente les labels
    def __init__(self, X, y):
        self.data = X
        self.label = y

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

In [ ]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 78, 10)
        )

    def forward(self, input):
        return self.model(input)

In [ ]:
""" Fonction qui va récuperer les données et qui va retourner les données et les labels """

def load_data(csv_file_name):
    labelled_dataset = pd.read_csv(f'/content/drive/MyDrive/projet_data_science_m2/projet_data/data/{csv_file_name}.csv', sep=';')
    print("labelled_dataset ...DONE")

    X = labelled_dataset.drop(columns=['label'])
    y = labelled_dataset['label']

    print(X.shape)
    print(y.shape)

    return X, y

""" Fonction qui va préparer les datasets en les divisant en fonction de la taille test_size passée en paramètre """

def prepare_datasets(test_size):
    X, y = load_data('labelled_dataset')
    # Transformation en tensor
    X_tensor = torch.tensor(X.values)
    y_tensor = torch.tensor(y.values)

    # Découpage des données pour la partie d'apprentissage et d'évaluation
    X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=test_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    train_dataset = CustomDataset(X_train, y_train)
    test_dataset = CustomDataset(X_test, y_test)

    return train_dataset, test_dataset

In [ ]:
def train(dataloader, model, loss_fn, optim, device):
    totalTrainLoss = 0
    trainCorrect = 0
    totalSamples = 0

    model.train()
    for (x, y) in dataloader:
        (x, y) = (x.to(device), y.to(device))
        pred = model(x.float())
        loss = loss_fn(pred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        totalTrainLoss += loss.item() * x.size(0)
        trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()
        totalSamples += x.size(0)

    # Calcule la perte moyenne et la précision moyenne en pourcentage
    train_loss = totalTrainLoss / len(dataloader.dataset)
    train_accuracy = (trainCorrect / totalSamples) * 100

    print("Accuracy : " + str(train_accuracy) + "% - Loss : " + str(train_loss) + "%")
    return train_loss, train_accuracy

Execution principale :

In [ ]:
if torch.cuda.is_available():
        device = 'cuda'
else:
        device = 'cpu'
print(device)
# paramètres pour l'apprentissage
learning_rate = 0.001
epochs = 50
batch_size = 64
loss_fn = nn.CrossEntropyLoss()


train_dataset, test_dataset = prepare_datasets(0.4)
# Création des DataLoader pour l'entrainement des données
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = CNN_Model().to(device)
print(model)

optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    print("Epochs : " + str(epoch + 1) + "/" + str(epochs))
    train(dataloader=train_dataloader, model=model, loss_fn=loss_fn, optim=optim, device=device)


cpu
labelled_dataset ...DONE
(30977, 78)
(30977,)
CNN_Model(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=4992, out_features=10, bias=True)
  )
)
Epochs : 1/50


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 64, 78, 1] to have 1 channels, but got 64 channels instead